В этом блокноте представлено построение и сохранение модели BERTopic на непредобработанных данных. На выходе имеем датасет link|num_of_topic - сохранение его. (почему -то так оно работает лучше, чем с предобработанными...однако, слова фильм, этот и т.п. всё же мешаются)

In [ ]:
!pip install transformers
!pip install bertopic

In [2]:
from bertopic import BERTopic

In [3]:
import numpy as np
import pandas as pd
import torch

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#загрузка полного датасета
mydf=pd.read_csv('/content/drive/MyDrive/model/final_df100.csv', index_col=0)
df = mydf[['id', 'review']].drop_duplicates(subset=['review'],ignore_index= True) # удаляю повторы комментариев

In [ ]:
df = df.sort_values(by = ['id'], ignore_index = True)
df = df.groupby('id').head(5) #датафрейм, в котором нет дубликатов и комментариев к одному фильму не более 5 !!!!!

# обучение и сохранение модели
docs_ = df["review"].to_list()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#вручную задаю количество тем 200 !!!!!
topic_model = BERTopic(language="russian", nr_topics = 200)

topics, _ = topic_model.fit_transform(docs_)

topic_model.save('/content/drive/MyDrive/model/mm3')

2024-04-06 16:54:54,474 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [4]:
# загрузка ранее рассчитанной модели BERTopic (Важно, делать это нужно на GPU) модель mm - на непредобработанных данных!!! просто эксперимент
topic_model = BERTopic.load('/content/drive/MyDrive/model/mm3')

In [11]:
# демонстрация модели
topic_model.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,21632,-1_не_что_на_но,"[не, что, на, но, как, то, это, фильм, из, по]","[Не редко бывает такой случай, когда у фильма ..."
1,0,196,0_рождество_рождества_санта_рождественская,"[рождество, рождества, санта, рождественская, ...",[В канун Рождества обычный работяга Келвин соб...
2,1,33,1_netflix_сериал_сериала_сервиса,"[netflix, сериал, сериала, сервиса, мэй, стрим...","[Разумеется, что интернет-гигант Netflix рано ..."
3,2,13,2_хоккея_хоккей_тарасова_боброва,"[хоккея, хоккей, тарасова, боброва, лучах, сла...","[С детства помню, какая всенародная любовь пок..."
4,3,28,3_канадский_отделе_канадского_nikita,"[канадский, отделе, канадского, nikita, дени, ...",[Канадский режиссер Дени Коте – этакая terra i...
5,4,51,4_чаплина_чарли_чаплин_чарльза,"[чаплина, чарли, чаплин, чарльза, essanay, бро...",[Студия ESSANAY представляет фильм Чарльза Чап...
6,5,460,5_он_индии_его_фильм,"[он, индии, его, фильм, очень, не, но, что, ин...",[Посмотрела ещё один современный фильм с Шахру...
7,6,53,6_футбол_футбола_матч_евро,"[футбол, футбола, матч, евро, сборной, команды...",[До недавнего времени футбольная история не зн...
8,7,26,7_asylum_the_студии_мега,"[asylum, the, студии, мега, студия, против, да...",[Дешевый антинаучный фантастический фильм-ката...
9,8,28,8_ирландии_ирландского_ирландский_ира,"[ирландии, ирландского, ирландский, ира, ирлан...","[Ирландский сериал, получивший звание культово..."


In [12]:
topic_model.visualize_topics()

In [ ]:
a = [0]*len(df['review']) # определение самой подходящей темы для каждого фильма из датасета
for i in range(len(df['review'])):
  a[i] = topic_model.find_topics(df['review'][i], top_n = 1)[0][0]

link_list = [None]*len(df['id']) # список ссылок в таблицу
for i in range(len(df['id'])):
  link_list[i] = 'https://www.kinopoisk.ru/film/'+df['id'][i].__str__()+'/'

top_df = df.assign(link = link_list, topic = a)
top_df_3 = top_df[top_df.topic > 1]


In [ ]:
# СОХРАНЕНИЕ ТАБЛИЦЫ
top_df_3.to_csv('top_df_3.csv')
!cp top_df_3.csv "drive/My Drive/model"

In [5]:
# загрузка таблицы top_df_3    ссылка на фильм|номер темы
top_df_2 = pd.read_csv('/content/drive/MyDrive/model/top_df_3.csv', index_col=0) # данные, пропущенные через модель

In [6]:
# функция, возвращающая ссылки на фильмы
def return_links(my_mes):
    my_topic = topic_model.find_topics(my_mes, top_n = 1)[0][0]
    return top_df_2[top_df_2.topic  == my_topic].sample(n=1).link #вывод подходящих ссылок


mymes = "обрести веру в любовь и настоящие искренние чувства"
return_links(mymes)

12304    https://www.kinopoisk.ru/film/77647/
Name: link, dtype: object

In [37]:
mymes = "про больницу и людей которые борятся с болезнью"
return_links(mymes)

19253    https://www.kinopoisk.ru/film/296488/
Name: link, dtype: object